In [1]:
import cv2
from os import makedirs
from os.path import isdir

# Flow
<ol>
    <li>Register face if not registered  </li>
    <li>Continue face recognition as requested by the server (Not actually implemented in this example)  </li>
    <li>Capture and process image</li>
 </ol>

# Chapter 1
Detect Face from aimage and save it

In [2]:
cv2.__version__

'4.7.0'

In [3]:
# 사용안함


# https://076923.github.io/posts/Python-opencv-2/
# https://scribblinganything.tistory.com/472

def capture():
    capture= cv2.VideoCapture(0)    # If the webcam is not installed, the CSI Cam becomes the #0 camera
    capture.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    while True:            # wait 33ms for key input, if key input detected, break Loop
        capture_result, frame = capture.read()        # grab() and retrieve() in one call
        cv2.imshow("VideoFrame", frame)
        if cv2.waitKey(1) == ord('c'):
            break

    capture.release()
    cv2.destroyAllWindows()
    
    return capture_result, frame

In [4]:
# 사용안함

# https://ai0.kr/m/41

def track_faces(capture, classifier, image_scale, max_faces):
    capture_copy = capture.copy()
    grayscale_capture = cv2.cvtColor(capture_copy, cv2.COLOR_BGR2GRAY)    # Apply Grayscale
    faces = classifier.detectMultiScale(grayscale_capture, image_scale, max_faces)


    return faces

In [5]:
# https://blog.naver.com/chandong83/221695462391

# function for consecutive capture
def detect_face(frame, classifier):
    grayscale_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = classifier.detectMultiScale(grayscale_frame, 1.3, 5)   # image, image_scale, max_faces
    if len(faces) == 0:
        return None
    for(x,y,w,h) in faces:
        cropped_face = frame[y:y+h, x:x+w]
    
    return cropped_face


In [6]:
# https://blog.naver.com/chandong83/221695462391

def capture_face(user_name, classifier):
    if not isdir(f"data/sub_data/{user_name}"):
        makedirs(f"data/sub_data/{user_name}")
    capture= cv2.VideoCapture(0)
    count = 0
    
    while True:
        result, frame = capture.read()
        if detect_face(frame, classifier) is not None:
            count+=1
            face = face = cv2.resize(detect_face(frame, classifier),(200,200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            cv2.imwrite(f"data/sub_data/{user_name}/user{count}.png",face)
            cv2.putText(face,str(count),(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),2)
            cv2.imshow('Face Cropper',face)
        else:
            print("Face not Found")
            pass
        # Press Enter or get 100images to break
        if cv2.waitKey(1)==13 or count==100:
            break

    capture.release()
    cv2.destroyAllWindows()
    print('Colleting Samples Complete!!!')

In [7]:
def main():
    classifier = cv2.CascadeClassifier("data/haarcascade_frontalface_default.xml")
    capture_face("lee", classifier)

In [8]:
import numpy as np
from os import listdir
from os.path import isdir, isfile, join

In [9]:
# Train User
# https://blog.naver.com/chandong83/221695462391

def train_face(user_name):
    data_path = "data/sub_data/" + user_name+ '/'
    face_pics = [f for f in listdir(data_path) if isfile(join(data_path,f))]
    training, labels = [], []
    
    for i, files in enumerate(face_pics):
        image_path = data_path + face_pics[i]
        images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        if images is None:
            continue    
            
        training.append(np.asarray(images, dtype=np.uint8))
        labels.append(i)
        
    if len(labels) == 0:
        print("There is no data to train.")
        return None
    
    labels = np.asarray(labels, dtype=np.int32)
    # 모델 생성
    model = cv2.face.LBPHFaceRecognizer_create()
    # 학습
    model.train(np.asarray(training), np.asarray(labels))
    print(name + " : Model Training Complete!!!!!")

    #학습 모델 리턴
    return model

In [10]:
# Train_multiple users
def trains():
    
    data_path = "data/sub_data/"
    # 폴더만 색출
    model_dirs = [f for f in listdir(data_path) if isdir(join(data_path,f))]
    
    #학습 모델 저장할 딕셔너리
    models = {}
    
    # 각 폴더에 있는 얼굴들 학습
    for model in model_dirs:
        
        print('model :' + model)
        # 학습 시작
        result = train_face(model)
        # 학습이 안되었다면 패스!
        if result is None:
            continue
        # 학습되었으면 저장
        print('model2 :' + model)
        models[model] = result

    # 학습된 모델 딕셔너리 리턴
    return models    



In [11]:
#얼굴 검출
def face_detector(img, size = 0.5):
        gray_scale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_classifier.detectMultiScale(gray_scale,1.3,5)
        if len(faces) == 0:
            return img,[]
        for(x,y,w,h) in faces:
            cv2.rectangle(img, (x,y),(x+w,y+h),(0,255,255),2)
            roi = img[y:y+h, x:x+w]
            roi = cv2.resize(roi, (200,200))
        return img,roi   #검출된 좌표에 사각 박스 그리고(img), 검출된 부위를 잘라(roi) 전달



In [12]:
def run(models):    
    #카메라 열기 
    capture = cv2.VideoCapture(0)
    
    while True:
        #카메라로 부터 사진 한장 읽기 
        capture_result, frame = capture.read()
        # 얼굴 검출 시도 
        image, face = face_detector(frame)
        try:            
            min_score = 999       #가장 낮은 점수로 예측된 사람의 점수
            min_score_name = ""   #가장 높은 점수로 예측된 사람의 이름
            
            #검출된 사진을 흑백으로 변환 
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            #위에서 학습한 모델로 예측시도
            for key, model in models.items():
                result = model.predict(face)                
                if min_score > result[1]:
                    min_score = result[1]
                    min_score_name = key
                    
            #min_score: 신뢰도 0에 가까울수록 정답에 가까움         
            if min_score < 500:
                
                confidence = int(100*(1-(min_score)/300))
                # 유사도 화면에 표시 
                display_string = str(confidence)+'% Confidence it is ' + min_score_name
            cv2.putText(image,display_string,(100,120), cv2.FONT_HERSHEY_COMPLEX,1,(250,120,255),2)
            #75 보다 크면 동일 인물로 간주해 UnLocked! 
            if confidence > 75:
                cv2.putText(image, "Unlocked : " + min_score_name, (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
                cv2.imshow('Face Cropper', image)
            else:
            #75 이하면 타인.. Locked!!! 
                cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
                cv2.imshow('Face Cropper', image)
        except:
            #얼굴 검출 안됨 
            cv2.putText(image, "Face Not Found", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)
            cv2.imshow('Face Cropper', image)
            pass
        if cv2.waitKey(1)==13:
            break
    capture.release()
    cv2.destroyAllWindows()

In [13]:
models = trains()
run(models)

model :lee


AttributeError: module 'cv2' has no attribute 'face'

In [ ]:
# 사용안함
def main():
    LINE_THICKNESS = 2
    LINE_COLOR = (255,0,0)
    classifier = cv2.CascadeClassifier("data/haarcascade_frontalface_default.xml")
    # Set Camera width and height (640 * 480)
    camera_width = 640
    camera_height = 480
    
    while True:
        option = int(input("1: Register\n2: Detect\nselect option: "))
       
        if option == 1:
            user_name = input("Input User Name: ").strip()
            capture_result, frame = capture()
            
            if capture_result == 1: 
                
                # Remove Image if there is a file with same name
                if os.path.isfile(f"data/sub_data/{user_name}.png"):
                    os.remove(f"data/sub_data/{user_name}.png")
                cv2.imwrite(f"data/sub_data/{user_name}.png", frame)
                img_without_grid = cv2.imread(f"data/sub_data/{user_name}.png")
                # Track Faces from 
                faces = track_faces(frame, classifier, 1.1, 3)
               
                if(len(faces)<1):
                    print("No Face detected, EXIT")
                    break
                # frame, classifier, 
                for i, (x, y, w, h) in enumerate(faces):
                    cv2.rectangle(img_without_grid, (x, y), (x+w,y+h), LINE_COLOR, LINE_THICKNESS)
                    cv2.putText(img_without_grid, str(i), (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0, 0), 2)
                
                # Show grid with Numbers
                # TODO: SEND THIS TO ANDROID CLIENT
                cv2.imshow("Enter the grid number you want to save", img_without_grid)
                cv2.waitKey(0)
                cv2.destroyAllWindows()
                
                grid_number = int(input().strip())
                # Assume you got the number from Android Client
                # TODO: GET A GRID NUMBER FROM ANDROID CLIENT
                
                x, y, w, h = faces[grid_number]
                print(f'y: {y}, x: {x}')
                
                # Crop Face and Apply Grayscale 
                grayscale_face = cv2.cvtColor(img_without_grid[y:y+h, x:x+w], cv2.COLOR_BGR2GRAY)
                cv2.imwrite(f"data/sub_data/{user_name}.png", grayscale_face)
#                 cv2.imwrite(f"data/sub_data/{user_name}.png", frame)
        elif(option == 2):
            print("Detect From Image")
            break
            
        else:
            break

    cv2.destroyAllWindows()
